In [11]:
import dill as pickle
import glob 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.spatial as spatial
import sys 
sys.path.append('..//CPN//')
import numpy as np 
import tqdm

In [12]:
results_folder = '..//simulations/multivariable_simulations//'

In [13]:
sim_results = glob.glob(results_folder+'*.simresults')

In [14]:
## A series of functions to parse the  simulation output
def get_num_echoes_heard(sim_data):
    '''
    '''
    num_echoes_heard = np.sum(sim_data[0])
    return(num_echoes_heard)

which_echo = {True: 1 , False:0}

def get_echoids(sim_data, heard=True):
    '''
    '''
    echo_indices = np.argwhere(sim_data[0]==which_echo[heard]).flatten()
    return(echo_indices)    

def get_echo_levels(sim_data, heard=True):
    '''
    '''
    echo_ids = get_echoids(sim_data, heard)
    
    echo_levels = sim_data[1]['target_echoes'].loc[echo_ids,'level']
    return(echo_levels)
    

def get_group_size(sim_data):
    ''' This function is necessary because of the
    stupid way I stored the parameter sets using classes
    '''
    num_bats_in_group = sim_data[0].size +1 
    return(num_bats_in_group)

def split_by_groupsize(df):
    all_subdfs = []
    group_sizes = np.unique(df['groupsize'])
    for each_groupsize in group_sizes:
        subdf = df[df['groupsize']==each_groupsize]
        all_subdfs.append(subdf)
    return(group_sizes, all_subdfs)



def get_individual_positions(sim_data):
    '''
    '''
    _, _b, geometry = sim_data
    positions = geometry['positions']
    return(positions)


def get_detection_distance(sim_data, heard=True):
    '''
    '''
    echo_inds = get_echoids(sim_data,heard)
    individuals_inds = echo_inds +1 # because focal individ is 0 index
    all_positions = get_individual_positions(sim_data)
    heard_individual_positions = all_positions[individuals_inds,:]
    focal_ind = all_positions[0,:]
    distances = spatial.distance
    
    positions_of_relevance = np.row_stack((focal_ind, heard_individual_positions))
    distances = spatial.distance_matrix(positions_of_relevance, 
                                        positions_of_relevance)[1:,0]
    return(distances)

In [15]:
### Load the data : this needs a decent amount of RAM !! and takes some time - remember this. 

In [16]:
all_simulation_data = []
for each in tqdm.tqdm(sim_results):
    with open(each, 'rb') as sim:
        output = pickle.load(sim)
    all_simulation_data.append(output)

100%|██████████| 8/8 [00:00<00:00, 20.22it/s]


In [17]:
sim_id, _ = all_simulation_data[0]

In [18]:
params_of_interest = ['interpulse_interval', 'echocall_duration', 'implement_shadowing','source_level','min_spacing',
                      'heading_variation', 'atmospheric_attenuation']

In [19]:
def extract_sim_parameters(sim_output, parameters):
    sim_id, _ = sim_output
    set_params = []
    for each in parameters:
        if each == 'source_level':
            set_params.append(sim_id['parameter_set'][each]['dBSPL'])
        else:
            set_params.append(sim_id['parameter_set'][each])
    return(set_params)
        

In [21]:
all_param = []
for each in all_simulation_data:
    all_param.append(str(extract_sim_parameters(each,params_of_interest)))

In [23]:
np.unique(all_param, return_counts=True)

(array(['[0.05, 0.001, False, 106, 1.0, 90, 0]',
        '[0.1, 0.001, False, 100, 0.5, 90, 0]',
        '[0.1, 0.0025, True, 100, 1.0, 90, 0]',
        '[0.1, 0.0025, True, 94, 1.0, 10, 0]'], dtype='|S37'),
 array([2, 2, 2, 2]))